## TensorFlow模型持久化

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 5.4 TensorFlow模型持久化

为了让训练结果可以复用，需要将训练得到的神经网络模型持久化。  
TF提供了一个API来保存和还原一个神经网络模型：tf.train.Saver类。

In [7]:
import tensorflow as tf

# 声明两个变量并计算它们的和
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
# 声明tf.train.Saver类用于保存模型
# 可以指定saver保存什么变量，如果只保存v1，那么下边要恢复result就会出错，因为缺少v2
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "C:/Users/74575/Desktop/pythonfile/model.ckpt")

TF模型一般会存在后缀为.ckpt的文件中，指定文件目录下会出现三个文件，因为计算图的结构和图上参数取值会分开保存：  
model.ckpt.meta保存计算图的结构  
model.ckpt保存每一个变量的取值  
checkpoint保存一个目录下所有的模型文件列表

加载这个已经保存的TF模型的方法：

In [8]:
with tf.Session() as sess:
    saver.restore(sess, "C:/Users/74575/Desktop/pythonfile/model.ckpt")
    print(result.eval())

INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/model.ckpt
[3.]


如果不希望重复定义图上的运算，可以直接加载持久化的图：tf.train.import_meta_graph

In [10]:
# 直接加载持久化的图
saver = tf.train.import_meta_graph(
    "C:/Users/74575/Desktop/pythonfile/model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess, "C:/Users/74575/Desktop/pythonfile/model.ckpt")
    # 通过张量的名称来获取张量
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/model.ckpt
[3.]


在声明tf.train.Saver类时可以提供一个列表来指定需要保存或者加载的变量，如saver=tf.train.Saver([v1])只加载v1。  
TF可以通过字典将模型保存时的变量名和需要加载的变量联系起来：

In [11]:
# 变量重命名
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
saver = tf.train.Saver({"v1": v1, "v2": v2})

这样做的目的之一是方便使用变量的滑动平均值。  
保存滑动平均模型的样例：

In [26]:
import tensorflow as tf
tf.reset_default_graph()

v = tf.Variable(0, dtype=tf.float32, name="v")
# 在没有申明滑动平均模型时只有一个变量v，所以以下语句只会输出“v:0”
for variables in tf.global_variables(): 
    print("variables.name:", variables.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())
# 在申明滑动模型之后，TF会自动生成一个影子变量
for variables in tf.global_variables(): 
    print("variables.name:", variables.name)

saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    # 保存的时候会将v:0和v/ExponentialMovingAverage:0这两个变量都存下来。
    saver.save(sess, "C:/Users/74575/Desktop/pythonfile/model.ckpt")
    print(sess.run([v, ema.average(v)]))

variables.name: v:0
variables.name: v:0
variables.name: v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [28]:
# 加载滑动平均模型
v = tf.Variable(0, dtype=tf.float32, name="v")

# 通过变量重命名将原来变量v的滑动平均值直接赋值给v。
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "C:/Users/74575/Desktop/pythonfile/model.ckpt")
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/model.ckpt
0.099999905


tf.train.ExponentialMovingAverage类提供了variables_to_restore函数来生成tf.train.Saver类所需要的变量重命名字典：

In [30]:
import tensorflow as tf
tf.reset_default_graph()

v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)

print("ema.variables_to_restore():", ema.variables_to_restore())
print("\n")
saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "C:/Users/74575/Desktop/pythonfile/model.ckpt")
    print(sess.run(v))

ema.variables_to_restore(): {'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}


INFO:tensorflow:Restoring parameters from C:/Users/74575/Desktop/pythonfile/model.ckpt
0.099999905


TF提供了convert_variables_to_constants函数，可以将计算图中的变量及其取值通过常量的方式保存，这样整个TF计算图可以统一存放在一个文件中：

In [31]:
# pb文件保存方法
import tensorflow as tf
tf.reset_default_graph()
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    # 导出当前计算图的GraphDef部分，只需要这一部分就可以完成从输入层到输出层的计算过程
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph_def, ['add'])
    # 将导出的模型存入文件
    with tf.gfile.GFile("C:/Users/74575/Desktop/pythonfile/combined_model.pb", "wb") as f:
           f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
ERROR! Session/line number was not unique in database. History logging moved to new session 115


In [32]:
# 加载pb文件
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename = "C:/Users/74575/Desktop/pythonfile/combined_model.pb"
    # 读取保存的模型文件，并将文件解析成对应的GraphDef Protocol Buffer
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # 将graph_def中保存的图加载到当前的图中
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([3.], dtype=float32)]


TensorFlow通过元图（MetaGraph）来记录计算图中节点的信息以及运行计算图中节点所需要的元数据。  
元图由MetaGraphDef Protocol Buffer定义，主要记录了6类信息。持久化原理及数据格式具体内容参见参考书5.4.2（p117-p126）